In [4]:
!nvidia-smi

Sun Jan  4 09:38:40 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 595.02                 Driver Version: 595.02         CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4070      WDDM  |   00000000:01:00.0  On |                  N/A |
| 58%   32C    P0             37W /  200W |    1509MiB /  12282MiB |      5%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
from huggingface_hub import login
login()

In [39]:
from huggingface_hub import whoami
whoami()

{'type': 'user',
 'id': '6884f9eeb4d898c5a7a7713f',
 'name': 'eleviacorp',
 'fullname': 'Rehan',
 'email': 'eleviacorporation@gmail.com',
 'emailVerified': True,
 'canPay': False,
 'billingMode': 'prepaid',
 'periodEnd': 1767225600,
 'isPro': False,
 'avatarUrl': 'https://cdn-avatars.huggingface.co/v1/production/uploads/no-auth/8SQ2Tu3BxC5t82VEmvBUK.png',
 'orgs': [],
 'auth': {'type': 'access_token',
  'accessToken': {'displayName': 'elenevread',
   'role': 'read',
   'createdAt': '2025-12-22T02:51:37.899Z'}}}

In [1]:
import torch
import convokit
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,   
    BitsAndBytesConfig
)

from peft import (
    LoraConfig,
    get_peft_model,
    PeftModel,
)


In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())


True
1


In [3]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
LORA_PATH = "./lora-llama3-rizz/roast/checkpoint-815" 

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb,
    device_map={"": 0}
)   

base_model.config.pad_token_id = tokenizer.eos_token_id
print("Model loaded successfully.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully.


In [2]:
model_id = "meta-llama/Llama-3.1-8B-Instruct"
LORA_PATH = "./lora-llama3-rizz/Training2/checkpoint-523" 

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb,
    device_map="auto"
)   

base_model.config.pad_token_id = tokenizer.eos_token_id

model = PeftModel.from_pretrained(
    base_model,
    LORA_PATH,
)

print("Model loaded successfully.")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully.


In [4]:
print("Model device:", next(base_model.parameters()).device)
print("Total params:", base_model.num_parameters())

Model device: cuda:0
Total params: 8043892736


In [3]:
import json
import os

CHAT_FOLDER = "instagram_chats"

all_messages = []

for filename in os.listdir(CHAT_FOLDER):
    if not filename.endswith(".json"):
        continue

    filepath = os.path.join(CHAT_FOLDER, filename)

    with open(filepath, "r", encoding="utf-8") as f:
        chat = json.load(f)

    participants = [p["name"] for p in chat.get("participants", [])]

    for msg in chat.get("messages", []):
        text = msg.get("content")
        if not text:
            continue

        all_messages.append({
            "chat_file": filename,
            "participants": participants,
            "sender": msg["sender_name"],
            "text": text,
            "timestamp": msg["timestamp_ms"]
        })

print(f"loaded {len(all_messages)} messages")

loaded 88451 messages


In [30]:
import json
import os

CHAT_FOLDER = "new_insta_chats"

all_messages = []

for filename in os.listdir(CHAT_FOLDER):
    if not filename.endswith(".json"):
        continue

    filepath = os.path.join(CHAT_FOLDER, filename)

    with open(filepath, "r", encoding="utf-8") as f:
        chat = json.load(f)

    participants = [p["name"] for p in chat.get("participants", [])]

    for msg in chat.get("messages", []):
        text = msg.get("content")
        if not text:
            continue

        all_messages.append({
            "chat_file": filename,
            "participants": participants,
            "sender": msg["sender_name"],
            "text": text,
            "timestamp": msg["timestamp_ms"]
        })

print(f"loaded {len(all_messages)} messages")

loaded 40961 messages


In [4]:
def fix_emoji(text):
    try:
        return text.encode("latin1").decode("utf-8")
    except:
        return text

In [5]:
import re

def is_real_text(text):
    t = text.lower().strip()

    junk_phrases = [
        "sent an attachment",
        "reacted",
        "you sent an attachment",
        "liked a message",
        "unsent a message",
    ]

    if any(j in t for j in junk_phrases):
        return False
    if len(t) < 3:
        return False
    if re.fullmatch(r"[^\w\s]+", t):
        return False
    return True

clean_messages = []

for m in all_messages:
    fixed_text = fix_emoji(m["text"])

    if is_real_text(fixed_text):
        m["text"] = fixed_text
        clean_messages.append(m)

print(f"Clean Messages: {len(clean_messages)}")

Clean Messages: 69410


In [6]:
MY_USERNAME = "Rehan Raza"

for m in clean_messages:
    m["is_me"] = (m["sender"] == MY_USERNAME)

In [7]:
clean_messages_sorted = sorted(clean_messages, key=lambda x: x["timestamp"])

In [8]:
def build_conversations(messages):
    conversation = []
    current = []
    last_chat = None

    for m in messages:
        if last_chat and m["chat_file"] != last_chat:
            if len(current) >= 2:
                conversation.append({"messages": current})
            current = []

        role = "assistant" if m["is_me"] else "user"
        current.append({"role": role, "content": m["text"]})
        last_chat = m["chat_file"]

    if len(current) >= 2:
        conversation.append({"messages": current})
    return conversation

conversations = build_conversations(clean_messages_sorted)
print(f"Built {len(conversations)} conversations")

Built 4270 conversations


In [9]:
with open("train.json", "w", encoding="utf-8") as f:
    json.dump(conversations, f, ensure_ascii=False, indent=2)

print("Saved All Conversations")

Saved All Conversations


In [10]:
def merge_consecutive_assistants(conversations):
    fixed_convs = []

    for conv in conversations:
        merged = []
        buffer = None

        for msg in conv["messages"]:
            if msg["role"] == "assistant":
                if buffer is None:
                    buffer = msg["content"]
                else:
                    buffer += " " + msg["content"]

            else:
                if buffer is not None:
                    merged.append({"role": "assistant", "content": buffer})
                    buffer = None
                merged.append(msg)

        if buffer is not None:
            merged.append({"role": "assistant", "content": buffer})

        fixed_convs.append({"messages": merged})

    return fixed_convs

In [11]:
conversations = merge_consecutive_assistants(conversations)
print("Assistant messages merged")

with open("train.json", "w", encoding="utf-8") as f:
    json.dump(conversations, f, ensure_ascii=False, indent=2)

print("Saved All Conversations")

Assistant messages merged
Saved All Conversations


In [12]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [4]:
import json

with open("train_1.json", "r", encoding="utf-8") as f:
    new_convs = json.load(f)

type(new_convs)

print(type(new_convs))
print(type(new_convs[0]))
print(new_convs[0].keys())


<class 'list'>
<class 'dict'>
dict_keys(['messages'])


In [13]:
def drop_leading_assistant(conversations):
    fixed = []
    for conv in conversations:
        msgs = conv["messages"]
        while msgs and msgs[0]["role"] == "assistant":
            msgs = msgs[1:]
        if len(msgs) >= 2:
            fixed.append({"messages": msgs})
    return fixed

conversations = drop_leading_assistant(conversations)

In [14]:
conversations = merge_consecutive_assistants(conversations)
print("Assistant messages merged")

with open("train.json", "w", encoding="utf-8") as f:
    json.dump(conversations, f, ensure_ascii=False, indent=2)

print("Saved All Conversations")

Assistant messages merged
Saved All Conversations


In [17]:
lengths = []

for conv in conversations:
    chat_text = tokenizer.apply_chat_template(
        conv["messages"],
        tokenize=False,
        add_generation_prompt=False,
    )

    token_ids = tokenizer(
        chat_text,
        truncation=True,
        max_length=1024,
    )["input_ids"]
    
    lengths.append(len(token_ids))


print("Min tokens:", min(lengths))
print("Max tokens:", max(lengths))
print("Avg tokens:", sum(lengths) // len(lengths))

Min tokens: 39
Max tokens: 1024
Avg tokens: 176


In [18]:
def tokenize_fn(example):
    chat_text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False,
    )

    out = tokenizer(
        chat_text,
        truncation=True,
        max_length=1024,
        padding="max_length",
    )

    
    out["labels"] = out["input_ids"].copy()
    return out


In [7]:
from convokit import Corpus, download
import random

corpus = Corpus(filename=download("subreddit-Cornell"))

rd_convos = list(corpus.iter_conversations())
random.shuffle(rd_convos)

rd_convos = rd_convos[:6000]

Dataset already exists at C:\Users\pc\.convokit\saved-corpora\subreddit-Cornell


In [8]:
def convo_to_messages(convo):
    messages = []
    for utt in convo.iter_utterances():
        role = "user" if len(messages) % 2 == 0 else "assistant"
        messages.append({
            "role": role,
            "content": utt.text.strip()
        })

    return messages

In [9]:
def is_good(messages):
    if len(messages) < 6:
        return False
    for m in messages:
        text = m["content"]
        if len(m["content"]) < 5:
            return False
        if text.lower() in ["lol", "ok", "yes", "no"]:
            return False

    for i in range(1, len(messages)):
        if messages[i]["role"] == messages[i-1]["role"]:
            return False
    
    return True

In [10]:
structured = []

for rd_convo in rd_convos:
    msgs = convo_to_messages(rd_convo)
    if is_good(msgs):
        structured.append({"messages": msgs})

structured = structured[:6000]

In [11]:
sample = structured[0]["messages"]

for msg in sample:
    print(msg["role"].upper(), ":", msg["content"])

USER : So my first semester of freshman year didn't go great, but I told myself some bullshit along the lines of I'm still adjusting to college blah blah, it's ok, I'll only go up from here, etc. Second semester was a bit better, although looking at my transcript, it's only because my classes had slightly higher median grades, I was still the same amount below the median as before... Anyway, I still felt decently optimistic and told myself third semester was gonna be the best one yet!!! Now that it's over, I've come to the conclusion that I'm simply stupid. 
It seems as though the difference between me genuinely trying to do better vs losing hope and aiming for bare minimum is pretty negligible, which isn't doing wonders for my motivation going forward...
Anyone have any tips for trying to suck a bit less/getting out of this mindset? Also any tips for making myself seem at least semi competent when applying for internships would be appreciated.
ASSISTANT : I actually feel exactly the s

In [12]:
def sanity_check(dataset, n=5):
    for i in range(n):
        conv = dataset[i]["messages"] 
        print("\n--- Conversation", i, "---")
        for m in conv:
            print(f'{m["role"]}: {m["content"]}')

In [36]:
sanity_check(structured, 3)


--- Conversation 0 ---
user: I'm completely redoing my schedule in Add/Drop, so I'm going to need to drop all my old ones in order to do the new ones.

Is there a way I can drop my old classes and add my new ones in one step, or do I have to delete then add?
assistant: You can “swap” them which will ensure that if you are not able to replace a class you already have with a desired one, the undesired one will remain. You can swap by code, class name, or directly from your shopping cart. The only downside is that you can’t swap more than one class at a time, so you can’t get them all done with a single click right as your appointment time opens.
user: "Swap".
assistant: The swap option is not there right now, unless I dont know where to look. Is it because add starts tomorrow 9 am?
user: It should be in the same place as where you can select add and drop. It was there for me before add/deop started
assistant: Found it but says I dont have enrollment appointment
user: Yep, that’s normal.

In [13]:
lengths = [len(c["messages"]) for c in structured]

print("Min Turns:", min(lengths))
print("Max Turns:", max(lengths))
print("Avg Turns:", sum(lengths)//len(lengths))

Min Turns: 6
Max Turns: 79
Avg Turns: 11


In [19]:
from datasets import Dataset

reddit_convo_ds = Dataset.from_list(structured)
print(reddit_convo_ds)

tokenized_dataset = reddit_convo_ds.map(
    tokenize_fn,
    remove_columns = ["messages"],
    batched = False,
)

Dataset({
    features: ['messages'],
    num_rows: 2092
})


Map:   0%|          | 0/2092 [00:00<?, ? examples/s]

In [20]:
print(tokenized_dataset)
print(tokenized_dataset[0].keys())
print(len(tokenized_dataset[0]["input_ids"]))


Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 2092
})
dict_keys(['input_ids', 'attention_mask', 'labels'])
1024


In [60]:
from datasets import load_dataset

roast_ds = load_dataset(
    "gus-gustavo/reddit_roastme",
    split="train"
)

print(roast_ds[0])

README.md:   0%|          | 0.00/342 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


data.csv:   0%|          | 0.00/26.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/397928 [00:00<?, ? examples/s]

{'comment': '$100,000 liberal arts degree, works at Starbucks. \n\nClaims to be an “old soul” yet has never gone more than an hour without WiFi.'}


In [64]:
import re

def is_valid_roast(text):
    text = text.strip()
    if len(text) < 15:
        return False
    if not re.search(r"[a-zA-Z]", text):
        return False
    if text.lower() in {"you", "no", "there", "go", "it", "100%"}:
        return False
    return True

roast_ds = roast_ds.filter(lambda x: is_valid_roast(x["comment"]))

print("Filtered roast samples:", len(roast_ds))
print(roast_ds[0])


Filter:   0%|          | 0/397928 [00:00<?, ? examples/s]

Filtered roast samples: 376117
{'comment': '$100,000 liberal arts degree, works at Starbucks. \n\nClaims to be an “old soul” yet has never gone more than an hour without WiFi.'}


In [67]:
INSULT_PROMPTS = [
    "you look dumb",
    "lmao you’re ugly",
    "you’re cringe",
    "bro you’re embarrassing",
    "why do you look like that",
    "nah you got no personality",
]

import random

def roast_to_chat(example):
    return {
        "messages": [
            {
                "role": "user",
                "content": random.choice(INSULT_PROMPTS),
            },
            {
                "role": "assistant",
                "content": example["comment"],
            }
        ]
    }

roast_chat = roast_ds.map(
    roast_to_chat,
    remove_columns=roast_ds.column_names
)

print(roast_chat[0])

Map:   0%|          | 0/376117 [00:00<?, ? examples/s]

In [73]:
from datasets import Dataset, concatenate_datasets

roast_chat = roast_chat.shuffle(seed=43).select(
    range(int(len(roast_chat) * 0.15))
)

print("Roast Sample Kept:", len(roast_chat))

Roast Sample Kept: 56417


In [21]:
from datasets import load_dataset

rizz_ds = load_dataset("Shaheer-ipynb/Rizz-Dataset", split="train")
print(rizz_ds[0])


README.md:   0%|          | 0.00/267 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


train-00000-of-00001.parquet:   0%|          | 0.00/15.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/545 [00:00<?, ? examples/s]

{'text': '• “Is your name Google? Because you’re everything I’ve been searching for.”'}


In [19]:
from datasets import Dataset

hf_dataset = Dataset.from_list(conversations)
insta_ds = hf_dataset
print(insta_ds[0])

{'messages': [{'content': 'Yes', 'role': 'user'}, {'content': 'Sure ig', 'role': 'assistant'}, {'content': 'Just add me', 'role': 'user'}, {'content': 'Done Briv', 'role': 'assistant'}, {'content': 'Yeah bro', 'role': 'user'}, {'content': 'Help-', 'role': 'assistant'}, {'content': 'Tell', 'role': 'user'}, {'content': 'Rate it? ☠', 'role': 'assistant'}, {'content': "It's actually good", 'role': 'user'}, {'content': '8.8/10', 'role': 'user'}, {'content': 'Bro u always look killer 💪💪', 'role': 'user'}, {'content': "First edit in a life- ☠ nah bro I don't-", 'role': 'assistant'}, {'content': 'Nah bro you are... ☠️', 'role': 'user'}, {'content': '☠️ (Reposted Nothing Changed-)', 'role': 'assistant'}, {'content': 'Bro u just need a sexy girl 🙂', 'role': 'user'}, {'content': 'Nah ☠ why-', 'role': 'assistant'}, {'content': "Cuz' u r sexy \nLHS=RHS", 'role': 'user'}, {'content': 'Whats LHS and RHS Doing in my life They were already bad enough in maths 😭😭', 'role': 'assistant'}, {'content': 'I m

In [20]:
from datasets import Dataset

insta_new_ds = Dataset.from_list(new_convs)
print(insta_new_ds[0])

NameError: name 'new_convs' is not defined

In [84]:
from datasets import Dataset, concatenate_datasets

def rizz_to_chat(example):
    return {
        "messages": [
            {"role": "user", "content": example["text"]},
            {"role": "assistant", "content": example["text"]},
        ]
    }

rizz_chat = rizz_ds.map(
    rizz_to_chat,
    remove_columns=rizz_ds.column_names
)

rizz_chat = rizz_chat.shuffle(seed=43).select(
    range(int(len(rizz_chat) * 0.3))
)

merged_ds = concatenate_datasets([
    insta_ds, 
    rizz_chat,
    roast_chat,
]).shuffle(seed=43)

merged_ds = merged_ds.shuffle(seed=42).select(
    range(int(len(merged_ds) * 0.3 ))
)

print("Final Dataset Size:", len(merged_ds))

Final Dataset Size: 17951


In [21]:
tokenized_dataset = insta_ds.map(
    tokenize_fn,
    remove_columns=["messages"],
    batched=False,
)

Map:   0%|          | 0/3258 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset = insta_new_ds.map(
    tokenize_fn,
    remove_columns = ["messages"],
    batched = False,
)

Map:   0%|          | 0/1194 [00:00<?, ? examples/s]

In [22]:
print(tokenized_dataset[0].keys())
print("input_ids len:", len(tokenized_dataset[0]["input_ids"]))
print("labels match:", tokenized_dataset[0]["input_ids"] == tokenized_dataset[0]["labels"])

dict_keys(['input_ids', 'attention_mask', 'labels'])
input_ids len: 1024
labels match: True


In [23]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    task_type="CAUSAL_LM",
)
    

In [24]:
model = get_peft_model(base_model, lora_config)
model.print_trainable_parameters()

model.enable_input_require_grads()
model.gradient_checkpointing_enable()
model.config.use_cache = False

trainable params: 13,631,488 || all params: 8,043,892,736 || trainable%: 0.1695


In [25]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./lora-llama3-rizz/Training3",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    bf16=True,
    logging_steps=25,
    save_steps=250,
    save_total_limit=2,
    report_to="none",
    optim="paged_adamw_8bit",
)


C:\DefaultPC\WebApplications\AiPractices\Elevia\ai-venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [26]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)


In [27]:
trainer.train()

Step,Training Loss
25,0.689300
50,0.472300
75,0.406800
100,0.435100
125,0.432400
150,0.364600
175,0.237800
200,0.312500
225,0.322500
250,0.399900


TrainOutput(global_step=814, training_loss=0.37736206324153215, metrics={'train_runtime': 7249.323, 'train_samples_per_second': 0.449, 'train_steps_per_second': 0.112, 'total_flos': 1.5040769345716224e+17, 'train_loss': 0.37736206324153215, 'epoch': 0.9993861264579497})

In [29]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

model_id = "meta-llama/Llama-3.1-8B-Instruct"

bnb = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb,
    device_map="auto",
)

model = PeftModel.from_pretrained(
    base_model,
    "./lora-llama3-rizz/roast/checkpoint-815"
)

model.eval()
print("Trained Model Loaded")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Trained Model Loaded


In [3]:
import random

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

with torch.no_grad():
    output = model.generate(
        inputs,
        max_new_tokens=200,
        temperature=1.5,
        top_p=0.95,
        do_sample=True,
        repetition_penalty=1.0
    )

generated_ids = output[0][inputs.shape[-1]:]

import re

text = tokenizer.decode(generated_ids, skip_special_tokens=True).strip()

# Remove role tokens even if glued to text
text = re.sub(r"(system|user|assistant)\s*", "", text, flags=re.IGNORECASE)

# Remove metadata lines
clean_lines = []
for line in text.split("\n"):
    line = line.strip()
    if not line:
        continue
    if line.startswith("Cutting Knowledge Date:"):
        continue
    if line.startswith("Today Date:"):
        continue
    clean_lines.append(line)

text = "\n".join(clean_lines)
bubbles = [line.strip() for line in text.split("\n") if line.strip()]

if len(bubbles) == 1:
    if random.random() < 0.9:
        # sentence-based split
        bubbles = re.split(r"[.!?]\s+", bubbles[0])


ASK_BACK_VARIANTS = ["wbu?", "what about you?", "hbu?", "you?"]
EMOJIS = ["😭😭", "💔🥀", "🥀", "😭🙏"]

new_bubbles = []

for i, bubble in enumerate(bubbles):
    modified = bubble

    # Emoji policy (per bubble)
    if not any(e in modified for e in EMOJIS):
            modified += " " + random.choice(EMOJIS)

    # Ask-back policy (ONLY on last bubble)
    if i == len(bubbles) - 1:
        if "?" not in modified and random.random() < 0.85:
            modified += " " + random.choice(ASK_BACK_VARIANTS)

    new_bubbles.append(modified)

bubbles = new_bubbles

print(bubbles)


NameError: name 'tokenizer' is not defined

In [32]:
# === CHAT WITH MEMORY + NEW USER MESSAGE ===

import json
import torch

# -------- CONFIG --------
CHAT_JSON_PATH = "chats.json"
MAX_TURNS = 30
MAX_NEW_TOKENS = 400

# -------- NEW USER MESSAGE --------
new_user_message = "HELLOOO"

# -------- LOAD CHAT --------
with open(CHAT_JSON_PATH, "r", encoding="utf-8") as f:
    chat_data = json.load(f)

# -------- NORMALIZE CHAT FORMAT --------
if isinstance(chat_data, list) and isinstance(chat_data[0], dict) and "role" in chat_data[0]:
    messages = chat_data

elif isinstance(chat_data, dict) and "messages" in chat_data:
    messages = chat_data["messages"]

elif isinstance(chat_data, list) and "messages" in chat_data[0]:
    messages = chat_data[0]["messages"]

else:
    raise ValueError("Unknown chats.json structure")

# -------- BUILD CONTEXT WINDOW --------
context_messages = messages[-(MAX_TURNS - 1):]

# ADD NEW USER MESSAGE
context_messages.append({
    "role": "user",
    "content": new_user_message
})

# -------- SYSTEM PROMPT --------
system_prompt = {
    "role": "system",
    "content": (
        "You are Elen. "
        "You text casually like a real person on Instagram. "
        "You reply naturally based on the conversation."
    )
}

prompt_messages = [system_prompt] + context_messages

# -------- TOKENIZE --------
inputs = tokenizer.apply_chat_template(
    prompt_messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# -------- GENERATE --------
with torch.no_grad():
    output = model.generate(
        inputs,
        max_new_tokens=MAX_NEW_TOKENS,
        temperature=0.9,
        top_p=0.85,
        do_sample=True,
        repetition_penalty=1.2,
    )

# -------- SLICE ONLY NEW TOKENS --------
generated_ids = output[0][inputs.shape[-1]:]

reply = tokenizer.decode(
    generated_ids,
    skip_special_tokens=True
).strip()

print("=== MODEL REPLY (RAW, NO LOGIC) ===")
print(reply)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


=== MODEL REPLY (RAW, NO LOGIC) ===
Helloo What 😿🥀 🫂👋🙏
